In [32]:
# based on https://www.kaggle.com/piantic/how-to-finetuning-models-pytorch-xla-tpu

import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [33]:
! ls ../input/cassava-disease

In [34]:
! rm -rf ./train
! unzip ../input/cassava-disease/train.zip > abc

In [35]:
! ls train

In [36]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
import numpy as np

In [37]:
transform = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

allimg = datasets.ImageFolder('./train', transform=transform)

validation_split = 0.25
dataset_size = len(allimg)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
np.random.seed(101)
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [38]:
train_loader = torch.utils.data.DataLoader(allimg, batch_size=16, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(allimg, batch_size=16,
                                                sampler=valid_sampler)

In [39]:
for images, labels in train_loader:
    print(images[0])
    print(labels[0])
    break

In [40]:
from torch import nn

class CustomResNext(nn.Module):
    def __init__(self, model_name, target_size, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, target_size)

    def forward(self, x):
        x = self.model(x)
        return x
    
    def freeze(self):
        # To freeze the residual layers
        for param in self.model.parameters():
            param.requires_grad = False

        for param in self.model.fc.parameters():
            param.requires_grad = True
    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.model.parameters():
            param.requires_grad = True

In [41]:
allimg.class_to_idx

In [42]:
m = CustomResNext('resnext50_32x4d', len(allimg.class_to_idx.keys()))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
m.to(device)

In [43]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduce = reduce

    def forward(self, inputs, targets):
        BCE_loss = nn.CrossEntropyLoss()(inputs, targets)

        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

In [44]:
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

optimizer = Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=1e-4, weight_decay=1e-6, amsgrad=False)
scheduler = ReduceLROnPlateau(optimizer, mode='min', verbose=True)
criterion = FocalLoss().to(device)
best_score = 0.
best_loss = np.inf
config_device = "GPU"

In [45]:
import math

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [46]:
from torch.cuda.amp import autocast, GradScaler

def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device):
    scaler = GradScaler()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to train mode
    model.train()
    start = end = time.time()
    global_step = 0
    for step, (images, labels) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with autocast():
            y_preds = model(images)
            loss = criterion(y_preds, labels)
            # record loss
            losses.update(loss.item(), batch_size)
#             if CFG.gradient_accumulation_steps > 1:
#                 loss = loss / CFG.gradient_accumulation_steps
            scaler.scale(loss).backward()
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1000)
            if (step + 1) % 1 == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                global_step += 1
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % 10 == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  #'LR: {lr:.6f}  '
                  .format(
                   epoch+1, step, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(train_loader)),
                   grad_norm=grad_norm,
                   #lr=scheduler.get_lr()[0],
                   ))
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to evaluation mode
    model.eval()
    trues = []
    preds = []
    start = end = time.time()
    for step, (images, labels) in enumerate(valid_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        # compute loss
        with torch.no_grad():
            y_preds = model(images)
        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)
        # record accuracy
        trues.append(labels.to('cpu').numpy())
        preds.append(y_preds.softmax(1).to('cpu').numpy())
#         if CFG.gradient_accumulation_steps > 1:
#             loss = loss / CFG.gradient_accumulation_steps
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % 10 == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(
                   step, len(valid_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(valid_loader)),
                   ))
    trues = np.concatenate(trues)
    predictions = np.concatenate(preds)
    return losses.avg, predictions, trues

In [47]:
from sklearn.metrics import accuracy_score
def get_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

In [48]:
valid_labels = []
for img, labels in validation_loader:
    valid_labels += labels.tolist()
valid_labels[:20]

In [ ]:
import time
for epoch in range(10):

    start_time = time.time()

    avg_loss = train_fn(train_loader, m, criterion, optimizer, epoch, scheduler, device)
    avg_val_loss, preds, _ = valid_fn(validation_loader, m, criterion, device)

    score = get_score(valid_labels, preds.argmax(1))

    elapsed = time.time() - start_time

    print(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
    print(f'Epoch {epoch+1} - Score: {score:.4f}')
    
    if score > best_score:
        best_score = score
        print(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
        torch.save({'model': m.state_dict(), 
                        'preds': preds},
                       'best_score.pth')

In [ ]:
! rm -rf ./test
! unzip ../input/cassava-disease/test.zip > xyz

In [ ]:
submission_img = datasets.ImageFolder('./test', transform=transform)
submission_loader = torch.utils.data.DataLoader(submission_img, batch_size=16)

In [ ]:
filenames = []
for path, idx in submission_img.imgs:
    filenames.append(path[path.index('0/test') + 2:])
filenames[:20]

In [ ]:
all_preds = []
for step, (images, labels) in enumerate(submission_loader):
    images = images.to(device)
    with torch.no_grad():
        all_preds += m(images).argmax(1).tolist()
all_preds[:20]

In [ ]:
! head -n 5 ../input/cassava-disease/sample_submission_file.csv

In [ ]:
rev_lookup = {}
for key in allimg.class_to_idx:
    rev_lookup[allimg.class_to_idx[key]] = key
rev_lookup

In [ ]:
import csv
with open('submission.csv', 'w') as opfile:
    wrt = csv.writer(opfile)
    wrt.writerow(['Category', 'Id'])
    for idx in range(0, len(filenames)):
        wrt.writerow([
            rev_lookup[all_preds[idx]],
            filenames[idx]
        ])

In [ ]:
! head -n 5 submission.csv